1. [가중치 초깃값] 가중치의 초깃값을 주는 기본적인 아이디어는 서로 다른 작은 숫자들을 할당하는 것이다. 얼마나 작게 줄 것인 지가 문제이다.

(1) 다음 표에 적절한 답을 적어보시오. common/multi_layer_net.py 참고.


std=0.01

권장되지 않지만, 활성화 함수로 시그모이드를 사용할 때 사용합니다.

In [ ]:
self.params['W' + str(idx)] = scale * np.random.randn(all_size_list[idx-1], all_size_list[idx])

Xavier

시그모이드를 사용할 때 효과적입니다.

In [ ]:
scale = np.sqrt(1.0 / all_size_list[idx - 1])  # sigmoid를 사용할 때의 권장 초깃값

He

렐루를 활성화 함수로 사용하는 경우 적합합니다.

In [ ]:
scale = np.sqrt(2.0 / all_size_list[idx - 1])  # ReLU를 사용할 때의 권장 초깃값

(2) 편향 b는 어떻게 초기화 하는가?

편향은 0으로 초기화합니다.

(3) [그림 6-14]에서, 왜 He 초깃값을 사용하는 것이 좋다고 말할 수 있는가?

활성화함수로 Relu를 사용하기 때문입니다.

(4) common/multi_layer_net_extend.py에서 가중치 초기화하는 함수를 찾아 코드를 옮겨 적으시오.



In [ ]:
def __init_weight(self, weight_init_std):
        """가중치 초기화
        
        Parameters
        ----------
        weight_init_std : 가중치의 표준편차 지정（e.g. 0.01）
            'relu'나 'he'로 지정하면 'He 초깃값'으로 설정
            'sigmoid'나 'xavier'로 지정하면 'Xavier 초깃값'으로 설정
        """
        all_size_list = [self.input_size] + self.hidden_size_list + [self.output_size]
        for idx in range(1, len(all_size_list)):
            scale = weight_init_std
            if str(weight_init_std).lower() in ('relu', 'he'):
                scale = np.sqrt(2.0 / all_size_list[idx - 1])  # ReLUを使う場合に推奨される初期値
            elif str(weight_init_std).lower() in ('sigmoid', 'xavier'):
                scale = np.sqrt(1.0 / all_size_list[idx - 1])  # sigmoidを使う場合に推奨される初期値
            self.params['W' + str(idx)] = scale * np.random.randn(all_size_list[idx-1], all_size_list[idx])
            self.params['b' + str(idx)] = np.zeros(all_size_list[idx])

2. [배치 정규화] 배치 정규화를 사용하면, [그림 6-19]에서 보듯이, 학습이 빨리 되어, 학습시 에폭 수를 줄일 수 있다(적은 에폭으로도 학습을 할 수 있다)고 알려져 있다.

(1) 배치 정규화의 기본 아이디어는 무엇인가?

각 층의 활성화값 분포가 적당히 퍼뜨리도록 강제하는 방법입니다.

(2) 배치 정규화는 두 가지 스탭(step)으로 구성되어 있다. 무엇인가? 필요한 수식을 적으시오.

$\gamma_B \leftarrow \frac{1}{m}\sum_{i=1}^mx_i$

$\sigma_B^2 \leftarrow \frac{1}{m}\sum_{i=1}^m(x_i-\gamma_b)^2$

$\hat{x_i} \leftarrow \frac{x_i-\gamma_B}{\sqrt{\sigma_B^2+\epsilon}}$

$y_i \leftarrow \gamma\hat{x_i}+\beta$

(3) 배치 정규화에도 학습하는 매개변수가 있는가?

$\beta, \gamma$를 학습합니다.

(4) common/multi_layer_net_extend.py에서 배치정규화 계층(layer)을 초기화하는 코드와 관련 매개변수의 기울기를 구하는 코드를 찾아 옮겨 적으시오

In [ ]:
#초기화
if self.use_batchnorm:
    self.params['gamma' + str(idx)] = np.ones(hidden_size_list[idx-1])
    self.params['beta' + str(idx)] = np.zeros(hidden_size_list[idx-1])
    self.layers['BatchNorm' + str(idx)] = BatchNormalization(self.params['gamma' + str(idx)], self.params['beta' + str(idx)])

#기울기 계산
if self.use_batchnorm and idx != self.hidden_layer_num+1:
    grads['gamma' + str(idx)] = numerical_gradient(loss_W, self.params['gamma' + str(idx)])
    grads['beta' + str(idx)] = numerical_gradient(loss_W, self.params['beta' + str(idx)])



(심화) 학습시와 테스트시 평균, 표준편차는 어떻게 구하는가? 즉, 어떤 값을 사용하는가?

배치단위의 평균과 표준편차가 모두 다름 테스트시에도 똑같이 평균과 표준편차를 구해야함 전체평균을 알고있을때 $(1-\frac{1}{n})M_{이전의 평균}+\frac{1}{n}x_n$